## 4 - Packed Padded Sequences, Masking, Inference and BLEU

> Trong notebook này, chúng ta sẽ cải tiến model ở notebook trước - áp dụng packed padded sequences và masking. Packed padded sequences có chức năng thông báo cho RNN biết và bỏ qua các padding tokens trong encoder. Trong khi đó, masking "ép" model ignore nhũng giá trị cụ thể - ví dụ như attention score của các padded elements. cả 2 kỹ thuật trên (packed padded sequences và masking) đều được sử dụng khá phổ biến trong NLP.

> Bên cạnh đó, chúng ta cũng xây dựng cách mô hình dự đoán - đầu vào là một câu, đầu ra sẽ là câu được dịch ở ngôn ngữ khác, đồng thời ta có thể quan sát điểm attention khi dịch mỗi từ.

> Cuối cùng, ta sẽ sử dụng BLEU metric để đánh giá chất lượng model dịch.

### Preparing Data

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchtext.legacy.datasets import Multi30k
from torchtext.legacy.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

In [2]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

> Import spacy và định nghĩa German và English tokenizers.

In [3]:
import spacy

spacy_de = spacy.load('de_core_news_sm')
spacy_en = spacy.load('en_core_web_sm')

In [4]:
def tokenize_de(text):
    """
    Tokenizes German text from a string into a list of strings (tokens) and reverses it
    """
    return [tok.text for tok in spacy_de.tokenizer(text)][::-1]

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

> Khi sử dụng *packed padded sequences*, chúng ta cần thông báo với PyTorch độ dài thực (non-padded) của sequences là gì. May mắn thay, trường `Field` trong TorchText cho phép ta sử dụng đối số `include_lengths` và chuyển `batch.src` thành một tupple với: phần tử đầu tiên trong tupple là một batch các source sentences đã được số hóa dưới dạng tensor, và phần tử thứ 2 là chiều dài thực (non-padded) của mỗi source sentence trong batch.

In [5]:
SRC = Field(tokenize = tokenize_de,
            init_token='',
            eos_token='',
            lower = True,
            include_lengths=True)

TRG = Field(tokenize = tokenize_en,
            init_token='',
            eos_token='',
            lower = True,
            include_lengths=True)

> Load data.

In [6]:
train_data, valid_data, test_data = Multi30k.splits(exts = ('.de', '.en'), fields = (SRC, TRG))

> Build Vocabulary.

In [7]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

> Tiếp theo, chúng ta xử lý iterators.

> Một điểm không rõ ràng về *packed padded sequences* là tất cả các phần tử trong batch cần được sắp xếp bởi chiều dài thực của câu (non-padded lengths) theo thứ tự giảm dần. Ta sẽ sử dụng 2 đối số trong iterator để xử lý vấn đề này: `sort_with_batch` thông báo cho iterator biết nội dung trong batch cần được sắp xếp, và `sort_key` là một hàm xác định cách mà iterator sắp xếp các elements trong batch. Cụ thể trong trường hợp này, ta sắp xếp theo chiều dài của source sentence, `src`.

In [8]:
BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator =\
    BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    device = device,
    sort_within_batch=True,
    sort_key=lambda x: len(x.src))

### Building the Model

##### Encoder

> Tiếp theo, ta xây dựng encoder.

> So với model trước, model lần này chỉ có chút thay đổi trong `forward` method. Bây giờ, nó nhận 2 đối số là: chiều dài sentence và sentence thay vì 1 đối số như trước.

> sau khi soure sentence (sẽ được tự động đệm trong iterator) được embedded, chúng ta có thể sử dụng `pack_padded_sequence` lên nó cùng với chiều dài thực của mỗi câu. Lưu ý là tensor chứa chiều dài của câu phải được chuyển về CPU. Sau đó, `packed_embedded` sẽ là *packed padded sequence* và được đưa vào RNN như bình thường. RNN sẽ trả về 2 đại lượng:  `packed_outputs` - một packed tensor chứa tất cả các hidden states từ sequence, và `hidden` - là final hidden state từ sequence. `hidden` là một tensor chuẩn và không được packed, điểm khác biệt duy nhất là inut là một packed sequence, tensor này thu được từ một **non-padded element** trong sequence.

> Sau đó, ta sẽ unpack `packed_outputs` với `pad_paced_sequence` - thứ sẽ trả về `outputs` và chiều dài của chúng (mỗi output có 1 chiều dài).

> Dimension đầu tiên trong `outputs` là chiều dài của padded sequence, tuy nhiên ta sử dụng *packed padded sequence* nên giá trị của các tensors đầu tiên trong `outputs` bằng 0.

In [ ]:
class Encoder(nn.Module):
    def __init__(self,
                input_dim, 
                emb_dim, 
                enc_hid_dim, 
                dec_hid_dim, 
                dropout):
        super().__init__()

        self.embedding = nn.Embedding(input_dim, emb_dim)

        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional=True)

        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)

        self.dropout = nn.Dropout(dropout)


    def forward(self, src, src_len):

        # src = [src len, batch size]
        # src_len = [batch size]

        embedded = self.dropout(self.embedding(src))

        # embedded = [src len, batch size, emb dim]

        # need to pack the sequence to feed to RNN, and put lengths to cpu
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, src_len.to('cpu'))

        packed_outputs, hidden = self.rnn(packed_embedded)
                                 
        # packed_outputs is a packed sequence containing all hidden states
        # hidden is now from the final non-padded element in the batch
            
        outputs, _ = nn.utils.rnn.pad_packed_sequence(packed_outputs) 
            
        # outputs is now a non-packed sequence, all hidden states obtained
        # when the input is a pad token are all zeros
            
        # outputs = [src len, batch size, hid dim * num directions]
        # hidden = [n layers * num directions, batch size, hid dim]
        
        # hidden is stacked [forward_1, backward_1, forward_2, backward_2, ...]
        # outputs are always from the last layer
        
        # hidden [-2, :, : ] is the last of the forwards RNN 
        # hidden [-1, :, : ] is the last of the backwards RNN
        
        # initial decoder hidden is final hidden state of the forwards and backwards 
        # encoder RNNs fed through a linear layer
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))
        
        # outputs = [src len, batch size, enc hid dim * 2]
        # hidden = [batch size, dec hid dim]
        
        return outputs, hidden

##### Attention

> Attetion module dùng để tính attention values của source sentence.

> Ở model trước, chúng ta cho phép attention module "chú ý" tới các padding tokens bên trong source sentence. Tuy nhiên, lần này chúng ta sẽ sử dụng *masking* để buộc attetion module chỉ hoạt động đối với các non-padding elements.